In [69]:
from pymongo import MongoClient
import pymongo
import snappy
import numpy as np
import json


client = MongoClient()
dbname = 'xamsdata0'

# Read the runs database and get the most recent 'baselines' run
cursor = client['run']['runs'].find().sort('start',pymongo.DESCENDING)
for doc in cursor:
    ini_name = doc['ini']['ini_name']
    if ini_name == 'baselines':
        break

# Read the collection name
collname = doc['ini']['mongo']['collection']

print('Reading baseline information from %s' % collname)

# Now connect to database containing data
d_1724 = {channel : [] for channel in range(8)}
d_1730 = {channel : [] for channel in range(8)}

cursor = client[dbname][collname].find()
for doc in cursor:
    channel = doc['channel']
    board = doc['module']
    d = snappy.decompress(doc['data'])
    w = np.fromstring(d, dtype='<i2')
    baseline = np.average(w)
    if board == 1730:
        d_1730[channel].append(baseline)
    elif board == 1724:
        d_1724[channel].append(baseline)
    else:
        print('Huh? What module is this: ', module)
        
        


baselines = {'1724': [np.median(d_1724[channel]) for channel in range(8)], 
             '1730' : [np.median(d_1730[channel]) for channel in range(8)]}
print('Baselines read:')
print(baselines)

with open('./baselines/baselines_%s.json' % collname, 'w') as outfile:
    json.dump(baselines, outfile, indent = 4)
    

Reading baseline information from 190205_094222
Baselines read:
{'1724': [8330.5514705882342, 8345.0882352941189, 8020.8063725490192, 8180.0220588235297, 8331.5857843137255, 8135.0882352941171, 8218.9093137254895, 8260.2377450980384], '1730': [8210.5380000000005, 8189.2749999999996, 7905.6959999999999, 8266.2749999999996, 6384.0519999999997, 8215.0360000000001, 8175.2719999999999, 8192.7270000000008]}
